In [22]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
import matplotlib.pyplot as plt
import math
import random
import itertools
import copy
from statistics import mean
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")
torch.manual_seed(1122)

Using cuda device


In [23]:
theta = [0.1, 1, 1.8, 2]

def gaussian(x, mu):
    return (1 / (0.3 * math.sqrt(2 * math.pi))) * (math.e ** ((-1/2) * (((x - mu) / 0.3)) ** 2))

def gaussian_mixture(x):
    return gaussian(x, theta[0]) + gaussian(x, theta[1]) + gaussian(x, theta[2]) + gaussian(x, theta[3])


In [24]:
class DE_NN(nn.Module):
    def __init__(self, NP, CR, F):
        super(DE_NN, self).__init__()
        lin1s = nn.ModuleList([nn.Linear(1, 4) for i in range(NP)])
        lin2s = nn.ModuleList([nn.Linear(4, 8) for i in range(NP)])
        lin3s = nn.ModuleList([nn.Linear(8, 4) for i in range(NP)])
        lin4s = nn.ModuleList([nn.Linear(4, 1) for i in range(NP)])
        self.layers = nn.ModuleList([lin1s, lin2s, lin3s, lin4s])
        ranges1 = [range(dim_size) for dim_size in lin1s[0].weight.shape]
        ranges2 = [range(dim_size) for dim_size in lin2s[0].weight.shape]
        ranges3 = [range(dim_size) for dim_size in lin3s[0].weight.shape]
        ranges4 = [range(dim_size) for dim_size in lin4s[0].weight.shape]
        self.idxs = [list(itertools.product(*ranges1)), list(itertools.product(*ranges2)), list(itertools.product(*ranges3)), list(itertools.product(*ranges4))]
        
        self.NP = NP
        self.CR = CR
        self.F = F
    def forward_i(self, X, layers): # a single pass
        for k in range(len(layers) - 1):
            X = torch.relu(layers[k](X))
        return layers[len(layers) - 1](X)
    def print_id(self, id, obj):
        if id == 0:
            print(obj)
    def step(self, id, X, Y, L): # forward pass with candidate i
        fx = L(self.forward_i(X, [l[id] for l in self.layers]), Y)
        agent_ids = random.sample(range(0, self.NP), 3) # how to efficiently reject self?
        y = [copy.deepcopy(l[id]).to(device) for l in self.layers]

        for k in range(len(self.layers)):
            R = tuple(random.randint(0, dim) for dim in self.layers[k][id].weight.shape)
            #print("indices", k, self.idxs[k])
            for i in self.idxs[k]:
                ri = random.random()
                if ri < self.CR or i == R:
                    y[k].weight[i] = self.layers[k][agent_ids[0]].weight[i] + self.F * (self.layers[k][agent_ids[1]].weight[i] - self.layers[k][agent_ids[2]].weight[i])
                    #print("OUT OF BOUNDS", id, "layer " + str(k) + " i " + str(i))
                else:
                    y[k].weight[i] = self.layers[k][id].weight[i]
            #self.print_id(id, y[k].weight)
        
        fy = L(self.forward_i(X, y), Y)
        if fy <= fx: 
            for k in range(len(self.layers)):
                self.layers[k][id] = y[k]
            return fy
        #self.print_id(id, fy)
        return fx
        

In [25]:
import torch.profiler

# 890 models or random candidates
# update individual parameters pi, 1 <= i <= 89 using unravel index but on each of the layers....
# 

epochs = 100

X = torch.randn(4000, 1).to(device)
Y = gaussian_mixture(X).to(device)

num_params = 89 #sum(p.numel() for p in model.parameters() if p.requires_grad)
NP = num_params
CR = 0.9
F = 0.8

model = DE_NN(NP, CR, F).to(device)
L = nn.MSELoss().to(device)

#with torch.autograd.profiler.profile(use_device='cuda') as prof:
for e in range(epochs):
    #for model in models:
    with torch.no_grad():
        f_lst = []
        for i in range(model.NP):
            f_lst.append(model.step(i, X, Y, L).item())
        print("min", min(f_lst), "max", max(f_lst), "avg", mean(f_lst))
            
        
            #print(f"Epoch [{e+1}/{epochs}], Loss: {loss.item():.4f}")
            #print(torch.cuda.utilization())
            #free, total = torch.cuda.mem_get_info(device)
            #mem_used_MB = (total - free) / 1024 ** 2
            #print(mem_used_MB)
            #print(torch.cuda.memory_summary())
    #nvidia-smi -lms 100 --query-gpu=index,gpu_name,memory.total,memory.used,memory.free,temperature.gpu,pstate,utilization.gpu,utilization.memory --format=csv

#print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))
#prof.export_chrome_trace("trace_with_grad_GPU.json")

min 0.22198030352592468 max 2.294090986251831 avg 0.9450842409991147
min 0.19093219935894012 max 1.9912303686141968 avg 0.8549477637148974
min 0.19093219935894012 max 1.9912303686141968 avg 0.7533473863025729
min 0.19093219935894012 max 1.9912303686141968 avg 0.6932411632511053
min 0.19093219935894012 max 1.8236595392227173 avg 0.6566605018765739
min 0.19093219935894012 max 1.628330945968628 avg 0.6188687712288974
min 0.19093219935894012 max 1.628330945968628 avg 0.5976691480433003
min 0.19093219935894012 max 1.628330945968628 avg 0.5903782881377788
min 0.16734735667705536 max 1.628330945968628 avg 0.5814126909114001
min 0.16734735667705536 max 1.5849899053573608 avg 0.5576640124736207
min 0.16734735667705536 max 1.316042423248291 avg 0.5259204704440041
min 0.16734735667705536 max 1.316042423248291 avg 0.4993716225530324
min 0.16734735667705536 max 1.316042423248291 avg 0.4985275159725982
min 0.16734735667705536 max 1.316042423248291 avg 0.4913923561238171
min 0.16734735667705536 max 1

In [26]:
test_X = torch.linspace(-1, 3, 1000)
test_Y = gaussian_mixture(test_X)
model_Y = model(test_X.reshape(-1, 1).to(device))
model_Y = model_Y.cpu()
plt.figure(figsize=(10, 6))
plt.plot(test_X.numpy(), test_Y.numpy(), label='Gaussian Mixture')
plt.plot(test_X.numpy(), model_Y.detach().numpy(), label='Predictions', color='red', linestyle='dotted')
plt.title('Gaussian Mixture Plot')
plt.xlabel('X')
plt.ylabel('Y')
plt.grid(True)
plt.show()

NotImplementedError: Module [DE_NN] is missing the required "forward" function